In [161]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
import import_ipynb
import import_librares # ноутбук с импортом библиотек
import func # ноутбук с функциями
import rus_ds_data # ноутбук с загрузкой и предварительной обрабткой русского датасета
#from ipynb.fs.full.func import tokenize_morphy, tokenize_lancaster, tokenize_snowball
from sklearn.metrics import accuracy_score
import string
from string import punctuation
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import SnowballStemmer
import re

In [162]:
from rus_ds_data import df_morphy, df_snowball, df_lancaster

In [163]:
from import_librares import CountVectorizer, TfidfVectorizer, plt, train_test_split\
,LogisticRegression, classification_report, roc_auc_score, MaxAbsScaler, GridSearchCV\
,cross_val_score, ppb, Word2Vec

In [164]:
df = pd.read_json('C:\\course_work\\feature_extraction_NLP\\data\\news_category\\News_Category_Dataset_v3.json', lines = True)

In [165]:
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [166]:
df = df.drop(['link', 'authors', 'date'], axis=1)

In [167]:
df.head()

,headline,category,short_description
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha..."
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to..."
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...


In [168]:
targets_names = df['category'].unique()

In [169]:
targets_dict = {i:name for name,i in enumerate(targets_names, 0)}

In [170]:
targets_dict

{'U.S. NEWS': 0,
 'COMEDY': 1,
 'PARENTING': 2,
 'WORLD NEWS': 3,
 'CULTURE & ARTS': 4,
 'TECH': 5,
 'SPORTS': 6,
 'ENTERTAINMENT': 7,
 'POLITICS': 8,
 'WEIRD NEWS': 9,
 'ENVIRONMENT': 10,
 'EDUCATION': 11,
 'CRIME': 12,
 'SCIENCE': 13,
 'WELLNESS': 14,
 'BUSINESS': 15,
 'STYLE & BEAUTY': 16,
 'FOOD & DRINK': 17,
 'MEDIA': 18,
 'QUEER VOICES': 19,
 'HOME & LIVING': 20,
 'WOMEN': 21,
 'BLACK VOICES': 22,
 'TRAVEL': 23,
 'MONEY': 24,
 'RELIGION': 25,
 'LATINO VOICES': 26,
 'IMPACT': 27,
 'WEDDINGS': 28,
 'COLLEGE': 29,
 'PARENTS': 30,
 'ARTS & CULTURE': 31,
 'STYLE': 32,
 'GREEN': 33,
 'TASTE': 34,
 'HEALTHY LIVING': 35,
 'THE WORLDPOST': 36,
 'GOOD NEWS': 37,
 'WORLDPOST': 38,
 'FIFTY': 39,
 'ARTS': 40,
 'DIVORCE': 41}

In [171]:
df = pd.DataFrame({'text' : [text for text in df['headline'] + ' ' + df['short_description']], 'label' : [targets_dict[label] for label in df['category']]})

In [172]:
df.head()

,text,label
0,Over 4 Million Americans Roll Up Sleeves For O...,0
1,"American Airlines Flyer Charged, Banned For Li...",0
2,23 Of The Funniest Tweets About Cats And Dogs ...,1
3,The Funniest Tweets From Parents This Week (Se...,2
4,Woman Who Called Cops On Black Bird-Watcher Lo...,0


In [173]:
nltk.download('stopwords')
nltk.download('punkt')
patterns = "[A-Za-z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
eng = "english"
sw = stopwords.words('english')
lancaster = LancasterStemmer()
#snowball = SnowballStemmer(language=eng)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chike\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chike\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [174]:
#ф-я для удаления символов пунктуации
def remove_punctuation(text):
    return ''.join([ch for ch in text if ch not in 
    string.punctuation])
#ф-я для удаления чисел
def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text]) 
#ф-я для удаления последовательностей
def remove_patterns(text):
    text = re.sub(patterns, ' ', text)
    return text
#ф-я для удаления не буквенных символов
def remove_notalpha(text):
    return ''.join(i if i.isalpha() else ' ' for i in text)

In [175]:
#ф-я токенизации с использованием Lancaster
def tokenize_lancaster(text):
    text = text.lower()
    text = remove_punctuation(text)
    sentences = sent_tokenize(text, language='english')
    all_words = []
    for sent in sentences:
        words = word_tokenize(sent, language='english')
        #print(words)
        for w in words:
            if w in sw:
                continue
            else:
                all_words.append(lancaster.stem(w))
    #print(*all_words)
    return ' '.join(all_words)

In [176]:
new_df = pd.DataFrame({"text" : [tokenize_lancaster(text) for text in df['text']], "label" : [label for label in df['label']]})

In [177]:
new_df.head()

,text,label
0,4 mil am rol sleev omicrontarget covid boost h...,0
1,am airlin fly charg ban lif punch flight atten...,0
2,23 funniest tweet cat dog week sept 1723 dog d...,1
3,funniest tweet par week sept 1723 accid put gr...,2
4,wom cal cop black birdwatch los lawsuit exempl...,0


In [178]:
# разбиение
x_train, x_test, y_train, y_test = train_test_split(new_df['text'], new_df['label'], test_size = 0.2, random_state = 45)

In [179]:
tfidf = TfidfVectorizer(min_df=5, max_df=1.0, ngram_range = (1, 1))

In [180]:
tfidf.fit(x_train)

TfidfVectorizer(min_df=5)

In [181]:
Xtrain = tfidf.transform(x_train)
Xtest = tfidf.transform(x_test)

In [183]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(Xtrain, y_train)

SVC(kernel='linear')

In [184]:
y_pred = clf.predict(Xtest)

In [185]:
accuracy_score(y_test, y_pred)

0.5973130339330883

In [186]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.40      0.11      0.18       246
           1       0.50      0.43      0.46      1014
           2       0.49      0.67      0.56      1747
           3       0.47      0.37      0.42       660
           4       0.55      0.27      0.36       218
           5       0.52      0.50      0.51       423
           6       0.67      0.72      0.69      1002
           7       0.57      0.73      0.64      3414
           8       0.68      0.84      0.75      7080
           9       0.36      0.28      0.31       528
          10       0.48      0.20      0.28       290
          11       0.36      0.34      0.35       206
          12       0.54      0.54      0.54       734
          13       0.61      0.50      0.55       437
          14       0.55      0.78      0.65      3670
          15       0.50      0.48      0.49      1181
          16       0.76      0.78      0.77      2033
          17       0.60    